In [44]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split
from pydub import AudioSegment

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

In [45]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [46]:
def load_data(test_size=0.2):
    X, y = [], []
    
    for file in glob.glob("/content/drive/MyDrive/Ravadess speech & song/Actor_*/*.wav"):
        # get the base name of the audio file
        basename = os.path.basename(file)
        sound = AudioSegment.from_wav(file)
        sound = sound.set_channels(1)
        sound.export(file, format="wav")
        print(basename)
        # get the emotion label
        emotion = int2emotion[basename.split("-")[2]]
        # we allow only AVAILABLE_EMOTIONS we set
        if emotion not in AVAILABLE_EMOTIONS:
            continue
        # extract speech features
        features = extract_feature(file, mfcc=True, chroma=False, mel=False)
        # add to data
        X.append(features)
        l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
        y.append(l[emotion])
    
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [47]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
X_train , X_val , y_train , y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=7)
print("[+] Number of training samples:", X_train.shape)
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape)
print("[+] Number of validation samples:", X_val.shape)

03-02-02-02-02-01-01.wav
03-02-02-01-02-01-01.wav
03-02-03-01-02-02-01.wav
03-02-03-01-02-01-01.wav
03-02-01-01-02-01-01.wav
03-02-01-01-01-01-01.wav
03-02-02-02-01-02-01.wav
03-02-02-02-01-01-01.wav
03-02-02-01-01-02-01.wav
03-02-03-01-01-02-01.wav
03-02-03-01-01-01-01.wav
03-02-01-01-02-02-01.wav
03-02-02-01-01-01-01.wav
03-02-01-01-01-02-01.wav
03-02-02-02-02-02-01.wav
03-02-02-01-02-02-01.wav
03-02-05-02-01-01-01.wav
03-02-05-01-01-02-01.wav
03-02-04-02-01-02-01.wav
03-02-05-02-01-02-01.wav
03-02-04-01-01-02-01.wav
03-02-05-01-02-01-01.wav
03-02-04-01-01-01-01.wav
03-02-03-02-02-01-01.wav
03-02-05-01-02-02-01.wav
03-02-04-02-02-02-01.wav
03-02-04-01-02-02-01.wav
03-02-03-02-02-02-01.wav
03-02-05-01-01-01-01.wav
03-02-04-01-02-01-01.wav
03-02-04-02-02-01-01.wav
03-02-03-02-01-01-01.wav
03-02-04-02-01-01-01.wav
03-02-03-02-01-02-01.wav
03-02-06-01-02-01-01.wav
03-02-05-02-02-01-01.wav
03-02-06-02-02-02-01.wav
03-02-06-02-02-01-01.wav
03-02-06-01-01-02-01.wav
03-02-05-02-02-02-01.wav


In [48]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_val= np.asarray(X_val)
y_val= np.asarray(y_val)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [49]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((789, 40), (789,), (329, 40), (329,))

In [50]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_valcnn = np.expand_dims(X_val, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)


In [51]:
x_traincnn.shape,x_testcnn.shape,x_valcnn.shape

((789, 40, 1), (329, 40, 1), (198, 40, 1))

In [52]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(40,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
opt = optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [53]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 40, 128)           768       
                                                                 
 activation_9 (Activation)   (None, 40, 128)           0         
                                                                 
 dropout_6 (Dropout)         (None, 40, 128)           0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 5, 128)           0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 5, 128)            82048     
                                                                 
 activation_10 (Activation)  (None, 5, 128)            0         
                                                      

In [54]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [55]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=10, epochs=500, validation_data=(x_valcnn, y_val))


Epoch 1/500
79/79 [==============================] - 2s 11ms/step - loss: 5.9034 - accuracy: 0.2383 - val_loss: 1.4165 - val_accuracy: 0.3485
Epoch 2/500
79/79 [==============================] - 1s 6ms/step - loss: 4.7888 - accuracy: 0.2712 - val_loss: 1.8062 - val_accuracy: 0.3131
Epoch 3/500
79/79 [==============================] - 0s 6ms/step - loss: 4.5155 - accuracy: 0.2725 - val_loss: 2.0282 - val_accuracy: 0.3232
Epoch 4/500
79/79 [==============================] - 0s 6ms/step - loss: 4.0952 - accuracy: 0.3131 - val_loss: 1.2861 - val_accuracy: 0.4343
Epoch 5/500
79/79 [==============================] - 0s 6ms/step - loss: 3.7038 - accuracy: 0.2991 - val_loss: 1.4502 - val_accuracy: 0.4293
Epoch 6/500
79/79 [==============================] - 1s 6ms/step - loss: 3.6720 - accuracy: 0.2738 - val_loss: 1.3620 - val_accuracy: 0.4141
Epoch 7/500
79/79 [==============================] - 1s 6ms/step - loss: 3.1837 - accuracy: 0.3004 - val_loss: 1.6498 - val_accuracy: 0.4141
Epoch 8/500


In [56]:
em=['happy','sad','neutral','angry']


In [57]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

11/11 [==============================] - 0s 5ms/step - loss: 0.5760 - accuracy: 0.7690
Restored model, accuracy: 76.90%


In [58]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
predictions = model.predict(x_testcnn)
classes_x=np.argmax(predictions,axis=1)
print(classification_report(y_test,classes_x))
print(confusion_matrix(y_test,classes_x))

              precision    recall  f1-score   support

         0.0       0.79      0.83      0.81        98
         1.0       0.78      0.77      0.78       105
         3.0       0.58      0.74      0.65        42
         4.0       0.86      0.71      0.78        84

    accuracy                           0.77       329
   macro avg       0.75      0.76      0.75       329
weighted avg       0.78      0.77      0.77       329

[[81  7  5  5]
 [ 5 81 15  4]
 [ 1  9 31  1]
 [15  7  2 60]]
